## Feature Evaluation
<hr style="height:1px;border:none;color:#333;background-color:#333;" />


## Table of Contents

* [1.1 Environment Setup](#env-setup)
    * [1.1.1 Importing Modules](#imports)
    * [1.1.2 Display Settings](#display-setup)
    * [1.1.3 Load Data](#read-file)
    * [1.1.4 Model Features](#features)
<br><br>
* [1.2 Evaluation of Features](#eval-section)
    * [1.2.1 Roof Material Evaluation](#eval-roof)
    * [1.2.2 Fence Quality Evaluation](#eval-fence)
    * [1.2.3 Overal Quality Evaluation](#eval-overall)

### 1.1 Environment Setup  <a class="anchor" id="env-setup"></a>

#### 1.1.1 Importing Modules<a class="anchor" id="imports"></a>

In [1]:
# import modules
from sklearn.linear_model import LinearRegression, Lasso, LassoCV
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
import warnings
import statsmodels.formula.api as smf
from scipy import stats
import math
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
warnings.filterwarnings("ignore")

#### 1.1.2 Display Settings<a class="anchor" id="display-setup"></a>

In [2]:
pd.set_option('display.max_columns', None) # display all the columns of a dataframe
pd.set_option('display.max_rows', 100)     # display 100 rows of a dataframe

#### 1.1.3 Load data<a class="anchor" id="read-file"></a>

In [4]:
# Load cleaned data - see "Data Cleansing.ipynb"
dataset = pd.read_csv('../ariel/Prep_Data.csv')
dataset_encoded = pd.read_csv('../ariel/Data_Encoded.csv')

#### 1.1.4 Features for Model (Including Target Variable)<a class="anchor" id="features"></a>

In [44]:
model_features = ['SalePrice','TotalBsmtSF', 'OverallQual', 'YearBuilt', 'log10_GrLivArea', 'OverallCond', 'log10_LotArea', 'BsmtUnfSF']

<br><br>
<hr style="height:1px;border:none;color:#333;background-color:#333;" />

### 1.5 Evaluating Overall Quality, Fence, and Roof<a class="anchor" id="eval-section"></a>

#### 1.2.1 Evaluation Roof Material<a class="anchor" id="eval-roof"></a>

<h4>Roof material</h4>
<p>&nbsp;</p>
<table style="border-collapse: collapse; width: 46.0227%; height: 162px;" border="1">
<thead>
<tr style="border-style: ridge; height: 18px;">
<td style="width: 3.50375%; height: 18px; border-style: ridge;">Abbreviation</td>
<td style="width: 9.18553%; height: 18px; border-style: ridge;">Material</td>
</tr>
</thead>
<tbody>
<tr style="height: 18px;">
<td style="width: 3.50375%; height: 18px; border-style: ridge;">ClyTile</td>
<td style="width: 9.18553%; height: 18px; border-style: ridge;">Clay or Tile</td>
</tr>
<tr style="height: 18px;">
<td style="width: 3.50375%; height: 18px; border-style: ridge;">CompShg</td>
<td style="width: 9.18553%; height: 18px; border-style: ridge;">Standard (Composite) Shingle</td>
</tr>
<tr style="height: 18px;">
<td style="width: 3.50375%; height: 18px; border-style: ridge;">Membran</td>
<td style="width: 9.18553%; height: 18px; border-style: ridge;">Membrane</td>
</tr>
<tr style="height: 18px;">
<td style="width: 3.50375%; height: 18px; border-style: ridge;">Metal</td>
<td style="width: 9.18553%; height: 18px; border-style: ridge;">Metal</td>
</tr>
<tr style="height: 18px;">
<td style="width: 3.50375%; height: 18px; border-style: ridge;">Roll</td>
<td style="width: 9.18553%; height: 18px; border-style: ridge;">Roll</td>
</tr>
<tr style="height: 18px;">
<td style="width: 3.50375%; height: 18px; border-style: ridge;">Tar&amp;Grv</td>
<td style="width: 9.18553%; height: 18px; border-style: ridge;">Gravel &amp; Tar</td>
</tr>
<tr style="height: 18px;">
<td style="width: 3.50375%; height: 18px; border-style: ridge;">WdShake</td>
<td style="width: 9.18553%; height: 18px; border-style: ridge;">Wood Shakes</td>
</tr>
<tr style="height: 18px;">
<td style="width: 3.50375%; height: 18px; border-style: ridge;">WdShngl</td>
<td style="width: 9.18553%; height: 18px; border-style: ridge;">Wood Shingles</td>
</tr>
</tbody>
</table>

In [6]:
roof_matl_dict = ({"ClyTile":"Clay or Tile","CompShg":"Standard (Composite) Shingle","Membran":"Membrane",
                   "Metal":"Metal","Roll":"Roll","Tar&Grv":"Gravel & Tar","WdShake":"Wood Shakes",
                   "WdShngl":"Wood Shingles",})

In [14]:
roofmatl_count = dataset.groupby(['RoofMatl']).agg({"RoofMatl":"count"})
roofmatl_count.columns = ['Number of Houses']
roofmatl_count['Percentage of Houses'] = roofmatl_count['Number of Houses']/dataset.shape[0]*100
roofmatl_count.reset_index(inplace=True)
roofmatl_count.columns=["Roof Material",'Number of Houses','Percentage of Houses']
roofmatl_count['Roof Material'].replace(roof_matl_dict, inplace=True)

In [26]:
roofmatl_count.style.applymap(lambda x: 'color: red' if (x<1.0) else 'color: black', subset=['Percentage of Houses']).hide_index()

Roof Material,Number of Houses,Percentage of Houses
Standard (Composite) Shingle,2381,98.673850
Membrane,1,0.041442
Metal,1,0.041442
Roll,1,0.041442
Gravel & Tar,15,0.621633
Wood Shakes,7,0.290095
Wood Shingles,7,0.290095


<ul>
<li>Approximately 99% of the houses in our dataset have Standard Composite Shingle on their roof. It is not possible to see a pattern to determine if a different roof materials would add or subtract value from the base price of the house.</li>
</ul>

We try looking for different patterns by grouping houses by neighborhood and size but it was not clear if the roof material made any difference on the price. However, after looking at the year built variable we were able to understand that in the last 30 year (since 1990) 99% of the houses have standard shingles.

In [95]:
# Evaluating the YearBuilt variable for each roof materical
df_roof_temp = dataset.groupby('RoofMatl').agg({"YearBuilt":["min","max","mean"]}).reset_index()
df_roof_temp.columns = ['Roof Material','Minimum(Year Built)', 'Maximum(Year Built)', 'Average(Year Built)']
df_roof_temp['Roof Material'].replace(roof_matl_dict, inplace=True)
df_roof_temp['Average(Year Built)'] = df_roof_temp['Average(Year Built)'].astype('int32')

In [141]:
df_roof_temp.style.hide_index()

Roof Material,Minimum(Year Built),Maximum(Year Built),Average(Year Built)
Standard (Composite) Shingle,1872,2010,1969
Membrane,1954,1954,1954
Metal,1969,1969,1969
Roll,1948,1948,1948
Gravel & Tar,1940,2004,1963
Wood Shakes,1967,1993,1979
Wood Shingles,1892,1994,1966


In [119]:
dataset[dataset['YearBuilt']>=1990].groupby('RoofMatl').agg({'RoofMatl':'count'})

,RoofMatl
RoofMatl,
CompShg,833
Tar&Grv,1
WdShake,1
WdShngl,1


Summary: it is clear from the data analyzed that the most prominent roof material is standard shingles. Should the house roof material requires replacement we recommend to use standard shingles. We are not able to determine from the data available if choosing other materials would increase or decrease the value of the house.

In [46]:
# Dataset of Model Features + dummified Roof Material features
df_roof = pd.concat([dataset_encoded[model_features],pd.get_dummies(dataset['RoofMatl'])], axis=1)
df_roof.rename(columns={"Tar&Grv":"TarGrv"}, inplace=True)

In [ ]:
# Model to evaluate roof material
mlr_model_roof = LinearRegression()
mlr_model_roof.fit(df_roof.drop(columns=['SalePrice']), np.log10(df_roof['SalePrice']))
mlr_model_roof.score(df_roof.drop(columns=['SalePrice']), np.log10(df_roof['SalePrice']))

In [72]:
# Create a list of observations to predict the SalePrice (only house with WdShae and WdShngl)
df_roof_test = df_roof[(df_roof['WdShngl']==1) | (df_roof['WdShake']==1)]
df_roof_test.reset_index(drop=True, inplace=True)

In [ ]:
# Predict SalePrice (7 WdShake and 7 WdShngl)
for i,y_hat in enumerate(mlr_model_roof.predict(df_roof_test.drop(columns='SalePrice'))):
    print(10**y_hat,' - ', df_roof_test.loc[i,'SalePrice'],' = ', 10**y_hat - df_roof_test.loc[i,'SalePrice'])

In [83]:
df_roof_test['WdShake']=0
df_roof_test['WdShngl']=0
df_roof_test['CompShg']=1

In [ ]:
# Predict SalePrice (7 WdShake and 7 WdShngl)
for i,y_hat in enumerate(mlr_model_roof.predict(df_roof_test.drop(columns='SalePrice'))):
    print(10**y_hat,' - ', df_roof_test.loc[i,'SalePrice'],' = ', 10**y_hat - df_roof_test.loc[i,'SalePrice'])

<br><br>
<hr>

#### 1.2.2 Fence Quality Evaluation<a class="anchor" id="eval-fence"></a>

<p><strong>Fence Quality</strong></p>
<table style="border-collapse: collapse; width: 44.472%; height: 103px;" border="1">
<thead>
<tr style="height: 18px;">
<td style="width: 8.16763%; border-style: ridge; text-align: center;"><em><strong>Rating</strong></em></td>
<td style="width: 8.26072%; height: 18px; border-style: ridge; text-align: center;"><em><strong>Quality</strong></em></td>
<td style="width: 38.401%; height: 18px; border-style: ridge; text-align: center;"><em><strong>Description</strong></em></td>
</tr>
</thead>
<tbody>
<tr style="height: 18px;">
<td style="width: 8.16763%; border-style: ridge; text-align: center;">4</td>
<td style="width: 8.26072%; height: 18px; border-style: ridge; text-align: center;">GdPrv</td>
<td style="width: 38.401%; height: 18px; border-style: ridge; text-align: left;">&nbsp; Good Privacy</td>
</tr>
<tr style="height: 18px;">
<td style="width: 8.16763%; border-style: ridge; text-align: center;">3</td>
<td style="width: 8.26072%; height: 18px; border-style: ridge; text-align: center;">MnPrv</td>
<td style="width: 38.401%; height: 18px; border-style: ridge; text-align: left;">&nbsp; Minimum Privacy</td>
</tr>
<tr style="height: 18px;">
<td style="width: 8.16763%; border-style: ridge; text-align: center;">2</td>
<td style="width: 8.26072%; height: 18px; border-style: ridge; text-align: center;">GdWo</td>
<td style="width: 38.401%; height: 18px; border-style: ridge; text-align: left;">&nbsp; Good Wood</td>
</tr>
<tr style="height: 13px;">
<td style="width: 8.16763%; border-style: ridge; text-align: center;">1</td>
<td style="width: 8.26072%; height: 13px; border-style: ridge; text-align: center;">MnWw</td>
<td style="width: 38.401%; height: 13px; border-style: ridge; text-align: left;">&nbsp; Minimum Wood/Wire</td>
</tr>
<tr style="height: 18px;">
<td style="width: 8.16763%; border-style: ridge; text-align: center;">0</td>
<td style="width: 8.26072%; height: 18px; border-style: ridge; text-align: center;">NA</td>
<td style="width: 38.401%; height: 18px; border-style: ridge; text-align: left;">&nbsp; No Fence</td>
</tr>
</tbody>
</table>

In [175]:
roof_fence_dict = ({0:"No Fence",1:"Minimum Wood/Wire",2:"Good Wood",3:"Minimum Privacy",4:"Good Privacy"})

In [183]:
df_fence = (dataset.groupby('Fence').agg(count=pd.NamedAgg(column="Fence", aggfunc="count"))/dataset.shape[0]*100).reset_index()
df_fence.columns=['Fence Rating','Percentage of Houses']
df_fence['Fence Rating'].replace(roof_fence_dict, inplace=True)
df_fence.style.hide_index()

Fence Rating,Percentage of Houses
No Fence,79.196022
Minimum Wood/Wire,0.414422
Good Wood,4.102777
Minimum Privacy,11.852466
Good Privacy,4.434314


<b>Trend on type of fences in last 20 (since 1990) years</b>

In [197]:
df_fence_temp = dataset_encoded.copy()
df_fence_temp = pd.concat([df_fence_temp.filter(model_features), dataset['Fence']],axis=1)

In [201]:
df_fence_temp.Fence.replace(roof_fence_dict, inplace=True)

In [205]:
df_fence_temp[df_fence_temp['YearBuilt']>=1990].groupby('Fence').agg({"Fence":"count"})

,Fence
Fence,
Good Privacy,25
Good Wood,4
Minimum Privacy,15
No Fence,792


In [211]:
df_fence_temp_test = pd.concat([df_fence_temp[df_fence_temp['Fence']!='No Fence'][model_features], pd.get_dummies(df_fence_temp[df_fence_temp['Fence']!='No Fence']['Fence'])], axis = 1)

In [213]:
# Model to evaluate roof material
mlr_model_fence = LinearRegression()
mlr_model_fence.fit(df_fence_temp_test.drop(columns=['SalePrice']), np.log10(df_fence_temp_test['SalePrice']))
mlr_model_fence.score(df_fence_temp_test.drop(columns=['SalePrice']), np.log10(df_fence_temp_test['SalePrice']))

0.8652079868146181

In [240]:
print(np.round(mlr_model_fence.coef_,4))
print(df_fence_temp_test.drop(columns=['SalePrice']).columns)

[ 1.000e-04  3.550e-02  1.500e-03  4.276e-01  2.610e-02  1.286e-01
 -0.000e+00  1.000e-02  7.000e-04  9.000e-04 -1.170e-02]
Index(['TotalBsmtSF', 'OverallQual', 'YearBuilt', 'log10_GrLivArea',
       'OverallCond', 'log10_LotArea', 'BsmtUnfSF', 'Good Privacy',
       'Good Wood', 'Minimum Privacy', 'Minimum Wood/Wire'],
      dtype='object')


In [222]:
df_fence_temp_test = pd.concat([df_fence_temp_test[df_fence_temp_test['Minimum Wood/Wire']==1].head(5),
           df_fence_temp_test[df_fence_temp_test['Good Wood']==1].head(5),
           df_fence_temp_test[df_fence_temp_test['Good Privacy']==1].head(5),
           df_fence_temp_test[df_fence_temp_test['Minimum Privacy']==1].head(5)],
          axis=0)
df_fence_temp_test.reset_index(inplace=True)

In [229]:
df_fence_temp_test

,SalePrice,TotalBsmtSF,OverallQual,YearBuilt,log10_GrLivArea,OverallCond,log10_LotArea,BsmtUnfSF,Good Privacy,Good Wood,Minimum Privacy,Minimum Wood/Wire
0,141000,1092.0,5,1957,3.038223,5,4.171726,197.0,0,0,0,1
1,123000,864.0,5,1967,2.954243,5,3.894205,95.0,0,0,0,1
2,110000,720.0,5,1947,3.108565,7,3.544068,408.0,0,0,0,1
3,145000,588.0,6,1958,3.056142,8,4.020444,81.0,0,0,0,1
4,127500,854.0,6,1954,3.106531,5,3.857332,181.0,0,0,0,1
5,136000,624.0,5,1991,3.105169,5,4.020154,166.0,0,1,0,0
6,72000,432.0,4,1925,2.827369,4,3.931153,432.0,0,1,0,0
7,153000,450.0,6,1978,3.014521,8,3.886660,0.0,0,1,0,0
8,167000,1277.0,7,1970,3.106191,5,4.026452,499.0,0,1,0,0
9,138500,1052.0,5,1968,3.022016,5,3.924279,36.0,0,1,0,0


In [233]:
df_fence_temp_test['Minimum Wood/Wire']=1
df_fence_temp_test['Minimum Privacy']=0
df_fence_temp_test['Good Wood']=0
df_fence_temp_test['Good Privacy']=0
for i, coef_ in enumerate(mlr_model_fence.predict(df_fence_temp_test.drop(columns=['SalePrice']))):
    print(10**coef_," - ",df_fence_temp_test.loc[i,'SalePrice']," = ", 10**coef_ - df_fence_temp_test.loc[i,'SalePrice'])

132486.19034309805  -  141000  =  -8513.809656901954
112295.53448843646  -  123000  =  -10704.46551156354
117293.73500828457  -  110000  =  7293.735008284566
154103.40489671077  -  145000  =  9103.404896710766
132607.45213476956  -  127500  =  5107.452134769555
139400.18191672783  -  136000  =  3400.18191672783
67098.48580349308  -  72000  =  -4901.51419650692
149418.30160677287  -  153000  =  -3581.698393227125
168696.9532606513  -  167000  =  1696.9532606512948
126707.93207161517  -  138500  =  -11792.06792838483
150036.57065003843  -  140000  =  10036.570650038426
245661.4769539346  -  235000  =  10661.476953934587
143157.65072578765  -  160500  =  -17342.34927421235
384087.439403482  -  468000  =  -83912.56059651799
215308.41157107372  -  257000  =  -41691.588428926276
184565.48153812185  -  186000  =  -1434.5184618781495
100913.61852029126  -  109500  =  -8586.38147970874
103701.79157595118  -  110000  =  -6298.208424048818
152364.18100151373  -  151500  =  864.1810015137307
12944

In [234]:
df_fence_temp_test['Minimum Wood/Wire']=0
df_fence_temp_test['Minimum Privacy']=1
df_fence_temp_test['Good Wood']=0
df_fence_temp_test['Good Privacy']=0
for i, coef_ in enumerate(mlr_model_fence.predict(df_fence_temp_test.drop(columns=['SalePrice']))):
    print(10**coef_," - ",df_fence_temp_test.loc[i,'SalePrice']," = ", 10**coef_ - df_fence_temp_test.loc[i,'SalePrice'])

136380.53414841022  -  141000  =  -4619.465851589775
115596.38733933924  -  123000  =  -7403.612660660758
120741.50665262363  -  110000  =  10741.506652623633
158633.1724044251  -  145000  =  13633.172404425102
136505.36035012323  -  127500  =  9005.360350123228
143497.75792446817  -  136000  =  7497.757924468169
69070.80134715814  -  72000  =  -2929.198652841864
153810.35360672622  -  153000  =  810.3536067262175
173655.6884556503  -  167000  =  6655.688455650292
130432.42780259773  -  138500  =  -8067.5721974022745
154446.7962589726  -  140000  =  14446.796258972608
252882.53334103327  -  235000  =  17882.533341033268
147365.67504019543  -  160500  =  -13134.324959804566
395377.43526241335  -  468000  =  -72622.56473758665
221637.2597073361  -  257000  =  -35362.74029266389
189990.66161040773  -  186000  =  3990.6616104077257
103879.90749077495  -  109500  =  -5620.0925092250545
106750.03704649972  -  110000  =  -3249.962953500275
156842.82517490338  -  151500  =  5342.825174903381
1

<br><br><br><br>
<hr>
<hr>

In [148]:
# Dataset of Model Features + dummified Fence feature
df_fence = pd.concat([dataset_encoded[model_features],pd.get_dummies(dataset['Fence'])], axis=1)


In [149]:
# Model to evaluate roof material
mlr_model_fence = LinearRegression()
mlr_model_fence.fit(df_fence.drop(columns=['SalePrice']), np.log10(df_fence['SalePrice']))
mlr_model_fence.score(df_fence.drop(columns=['SalePrice']), np.log10(df_fence['SalePrice']))

0.9108802494011978

In [152]:
# Select 5 houses with no fence to test
df_fence_test = df_fence[df_fence[0]==1].head(5)

In [161]:
for i, coef_ in enumerate(mlr_model_fence.predict(df_fence_test.drop(columns=['SalePrice']))):
    print(10**coef_," - ",df_fence_test.loc[i,'SalePrice']," = ", 10**coef_ - df_fence_test.loc[i,'SalePrice'])

113645.8067445226  -  126000  =  -12354.193255477396
129578.91342986427  -  139500  =  -9921.086570135725
129653.89508722632  -  124900  =  4753.895087226323
92705.56503288145  -  114000  =  -21294.43496711855
241319.7558071002  -  227000  =  14319.755807100213


In [162]:
df_fence_test[0]=0
df_fence_test[1]=1
for i, coef_ in enumerate(mlr_model_fence.predict(df_fence_test.drop(columns=['SalePrice']))):
    print(10**coef_," - ",df_fence_test.loc[i,'SalePrice']," = ", 10**coef_ - df_fence_test.loc[i,'SalePrice'])

109362.68676308492  -  126000  =  -16637.313236915084
124695.30136195851  -  139500  =  -14804.69863804149
124767.45708632673  -  124900  =  -132.54291367327096
89211.64766490128  -  114000  =  -24788.352335098723
232224.81867196588  -  227000  =  5224.818671965884


In [163]:
df_fence_test[1]=0
df_fence_test[2]=1
for i, coef_ in enumerate(mlr_model_fence.predict(df_fence_test.drop(columns=['SalePrice']))):
    print(10**coef_," - ",df_fence_test.loc[i,'SalePrice']," = ", 10**coef_ - df_fence_test.loc[i,'SalePrice'])

112589.02973854999  -  126000  =  -13410.970261450013
128373.97661702233  -  139500  =  -11126.02338297767
128448.26103008086  -  124900  =  3548.261030080859
91843.5085060385  -  114000  =  -22156.491493961497
239075.75599462  -  227000  =  12075.755994620005


In [164]:
df_fence_test[2]=0
df_fence_test[3]=1
for i, coef_ in enumerate(mlr_model_fence.predict(df_fence_test.drop(columns=['SalePrice']))):
    print(10**coef_," - ",df_fence_test.loc[i,'SalePrice']," = ", 10**coef_ - df_fence_test.loc[i,'SalePrice'])

112465.2535843532  -  126000  =  -13534.746415646805
128232.84708458473  -  139500  =  -11267.152915415267
128307.04983214721  -  124900  =  3407.0498321472114
91742.53919937323  -  114000  =  -22257.460800626766
238812.92508021012  -  227000  =  11812.925080210116


In [165]:
df_fence_test[3]=0
df_fence_test[4]=1
for i, coef_ in enumerate(mlr_model_fence.predict(df_fence_test.drop(columns=['SalePrice']))):
    print(10**coef_," - ",df_fence_test.loc[i,'SalePrice']," = ", 10**coef_ - df_fence_test.loc[i,'SalePrice'])

113389.20013058938  -  126000  =  -12610.79986941062
129286.33064863476  -  139500  =  -10213.669351365243
129361.14300112094  -  124900  =  4461.143001120945
92496.24044962312  -  114000  =  -21503.759550376883
240774.86772731334  -  227000  =  13774.867727313336


#### 1.2.3 Overall Quality Evaluation<a class="anchor" id="eval-overall"></a>

In [ ]:
dataset['OverallQual'].value_counts()